In [ ]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
conn = psycopg2.connect(
    dbname="mimiciv",
    user="anya",
    # password="your_password",
    host="localhost",  # or your DB server
    port="5432"
)

cur = conn.cursor()

cohort = pd.read_csv('data/8_final_cohort-demo_comorb.csv')
print(cohort.shape)
cohort.head(5)

# discharge notes
cur.execute("SELECT * FROM mimiciv_note.discharge WHERE subject_id IN %s", (tuple(cohort['subject_id']),))
results = cur.fetchall()
columns = [desc[0] for desc in cur.description]
discharge_notes = pd.DataFrame(results, columns=columns)
print(discharge_notes['hadm_id'].isin(cohort['hadm_id']).all())
print(discharge_notes.shape)

# radiology notes
cur.execute("SELECT * FROM mimiciv_note.radiology WHERE subject_id IN %s", (tuple(cohort['subject_id']),))
results = cur.fetchall()
columns = [desc[0] for desc in cur.description]
radiology_notes = pd.DataFrame(results, columns=columns)
print(radiology_notes['hadm_id'].isin(cohort['hadm_id']).all())
print(radiology_notes.shape)

# close connection
cur.close()
conn.close()

In [ ]:
rad_notes = radiology_notes.dropna()
rad_notes = rad_notes[rad_notes['hadm_id'].isin(cohort['hadm_id'])]
print(rad_notes['hadm_id'].isin(cohort['hadm_id']).all())
print(rad_notes.shape)

In [ ]:
22691+7569

In [ ]:
notes = pd.concat([discharge_notes, rad_notes])
notes

In [ ]:
notes.to_csv('data/10_final_cohort_notes.csv', index=False, header=True)

In [ ]:
df = pd.merge(notes, cohort, how='inner', on=['subject_id', 'hadm_id'])
df

### EDA

In [ ]:
cases = df[df['case_status']==1]
ctrls = df[df['case_status']==0]

In [ ]:
# for cases, how many cases have 0 notes when they are not diagnosed with AD
# (i.e., how many patients have no ad=0)
ad_counts = cases.groupby('subject_id')['ad'].value_counts().unstack(fill_value=0)
ad_counts.columns = ['ad0', 'ad1']
ad_counts['ad0'].value_counts()

In [ ]:
df[df['case_status'] == 1]['subject_id'].value_counts().describe()

In [ ]:
df[df['case_status'] == 0]['subject_id'].value_counts().describe()

In [ ]:
counts = df.groupby(['case_status', 'subject_id']).size().reset_index(name='count')
freq = counts.groupby(['case_status', 'count']).size().reset_index(name='num_subjects')

plt.figure(figsize=(8, 5))
sns.boxplot(data=counts, x='case_status', y='count', palette='Set2')
plt.title('Distribution of Subject Occurrence Counts by Case Status')
# plt.yscale('log')
plt.xlabel('Case Status')
plt.ylabel('Occurrences per Subject')
plt.tight_layout()
plt.show()

In [ ]:
counts['many'] = counts['count'] > 1
pd.crosstab(counts['many'], counts['case_status'])

In [ ]:
duration = df.copy()
duration['admitdate'] = pd.to_datetime(duration[['admityear', 'admitmonth', 'admitday']].rename(
    columns={'admityear': 'year', 'admitmonth': 'month', 'admitday': 'day'}))
duration = duration.groupby('subject_id').agg(
    first_date = ('admitdate', 'min'),
    laste_date = ('admitdate', 'max'),
    duration_days = ('admitdate', lambda x: (x.max() - x.min()).days),
    case_status=('case_status', 'first'))
print(duration['case_status'].value_counts())

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=duration, x='case_status', y='duration_days', palette='Set2')
plt.title('Distribution of Day Span Between Discharge Summaries by Case Status')
# plt.yscale('log')
plt.xlabel('Case Status')
plt.ylabel('Span in Days')
plt.tight_layout()
plt.show()

In [ ]:
duration[duration['case_status'] == 0]['duration_days'].describe()

In [ ]:
duration[duration['case_status'] == 1]['duration_days'].describe()

In [ ]:
df.to_csv('data/11_final_cohort_alldata.csv', index=False, header=True)

In [ ]:
df.columns